# 4-1. 의사록_split(2,3)

In [7]:
#의사록 Section2, 3만 가져와서 합치기

import pandas as pd 

minutes_section = pd.read_csv('./Project/minutes_section_final_date.csv')

minutes_section['section2'] = minutes_section['section2'].fillna("0")
minutes_section['section3'] = minutes_section['section3'].fillna("0")

total = minutes_section.loc[(minutes_section['section2']!= '0') | (minutes_section['section3']!= '0')]

sec = pd.DataFrame()

sec['date'] = total['date']
sec['section2'] = total['section2']
sec['section3'] = total['section3']
sec['section2_3'] = sec['section2'] +  sec['section3']

sec.to_csv('./Project/minutes_section_2and3_total.csv', header=True, index=False)


#Section2&3 문장별로 자르기

sec = pd.read_csv('./Project/minutes_section_2and3_total.csv')

split_sec=pd.DataFrame(columns = ['date','sec2_sec3_split'])

new = pd.DataFrame();
count=0
p=0

for i in range(len(sec)):
    
    sec['section2_3'][i] = sec['section2_3'][i].replace(str('[]'), "")
    sec['section2_3'][i] = sec['section2_3'][i].replace(str(']'), "")
    sec['section2_3'][i] = sec['section2_3'][i].replace(str('['), "")

    new = sec['section2_3'][i].split(str("@@@"))
    
    new_len=len(new)
    count=count+i
    
    for j in range(new_len):
        split_sec.loc[i+p,'sec2_sec3_split'] = new[j]
        split_sec.loc[i+p,'date'] =  sec['date'][i]
        p=p+1
        count=count+j


split_sec.to_csv('./Project/minutes_section_2and3_sentence_split.csv', index = False)

split_sec.head()

,date,sec2_sec3_split
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...
4,2005.06.09,다른 일부 위원은 최근 가계대출의 월평균 증가액이 2003년과 지난해 실적 을 크...


# 4-2. 의사록_토큰화

In [1]:
#의사록 문장별로 토큰화

import pandas as pd
from ekonlpy.sentiment import MPCK
mpck = MPCK()


split = pd.read_csv('./minutes_section_2and3_sentence_split.csv')


split['tokens']=""
split['ngrams']=""

for i, row in split.iterrows() :
    
        aa = row['sec2_sec3_split']
        tokens = mpck.tokenize(aa)
        row['tokens'] = ','.join(tokens)
        ngrams =  mpck.ngramize(tokens)
        row['ngrams'] = ','.join(ngrams)

split.to_csv('./Project/sec2_sec3_sentence_split_token_ngram.csv',index=False)
split.head()

,date,sec2_sec3_split,tokens,ngrams
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/..."
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...,"환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/N...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일..."
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...,"유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/..."
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...,"금리/NNG,수준/NNG,유지/NNG,되/XSV,금리/NNG,상승/NNG,하/XSV...","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/NNG"
4,2005.06.09,다른 일부 위원은 최근 가계대출의 월평균 증가액이 2003년과 지난해 실적 을 크...,"가계/NNG,대출/NNG,증가/NNG,액/NNG,실적/NNG,크/VA,넘/VV,주택...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN..."


In [2]:
#tokens&ngrams 합치기

split['tokens']=split['tokens'].fillna("0")
split['ngrams']=split['ngrams'].fillna("0")


if(str(split['ngrams'])== "0"):
    split['ngrams and tokens'] = split['tokens']
elif(str(split['tokens'])== "0"):
    split['ngrams and tokens'] = split['ngrams']
else:
    split['ngrams and tokens'] = split['ngrams'] + "," + split['tokens']



for i in range(len(split)):
    b = split['ngrams and tokens'][i]
    b= b.replace("0,", "")
    b= b.replace(",0", "")
    split['ngrams and tokens'][i]=b
    
split.to_csv('./Project/sec2_sec3_sentence_split_tokens_ngrams_merge.csv', header=True, index=False)
split.head()

,date,sec2_sec3_split,tokens,ngrams,ngrams and tokens
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/..."
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...,"환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/N...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일..."
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...,"유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/..."
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...,"금리/NNG,수준/NNG,유지/NNG,되/XSV,금리/NNG,상승/NNG,하/XSV...","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/NNG","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/N..."
4,2005.06.09,다른 일부 위원은 최근 가계대출의 월평균 증가액이 2003년과 지난해 실적 을 크...,"가계/NNG,대출/NNG,증가/NNG,액/NNG,실적/NNG,크/VA,넘/VV,주택...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN..."


# 4-3. 의사록_매비count

In [3]:
import pandas as pd
from collections import defaultdict

n_t_merge = pd.read_csv('./Project/sec2_sec3_sentence_split_tokens_ngrams_merge.csv')
dicts = pd.read_csv('./Project/total_label_re.csv')

# 열 생성
n_t_merge['매파count'] = ""
n_t_merge['비둘기파count'] = ""


# 초기값
a_cnt = 0
b_cnt = 0


# 사전 dictionary로 형식변경
dic = defaultdict(lambda: 0)

for i, row in dicts.iterrows() :
    dic[row["ngrams"]] = row["label"]


for i in range(len(n_t_merge)):
    n_t_dic = n_t_merge['ngrams and tokens'][i].split(',')

    for j in range(len(n_t_dic)):
        n_t = n_t_dic[j]
        
        if dic[n_t] == 1:
            a_cnt = a_cnt + 1
        elif dic[n_t] == -1:
            b_cnt = b_cnt + 1
        else:
            continue
            
            
    n_t_merge['매파count'][i] = a_cnt
    n_t_merge['비둘기파count'][i] = b_cnt
    a_cnt = 0
    b_cnt = 0


n_t_merge.to_csv('./Project/total_label_matching.csv', index = False)
n_t_merge.head()



,date,sec2_sec3_split,tokens,ngrams,ngrams and tokens,매파count,비둘기파count
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...",3,1
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...,"환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/N...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...",1,2
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...,"유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...",1,3
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...,"금리/NNG,수준/NNG,유지/NNG,되/XSV,금리/NNG,상승/NNG,하/XSV...","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/NNG","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/N...",1,2
4,2005.06.09,다른 일부 위원은 최근 가계대출의 월평균 증가액이 2003년과 지난해 실적 을 크...,"가계/NNG,대출/NNG,증가/NNG,액/NNG,실적/NNG,크/VA,넘/VV,주택...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN...",0,0


# 4-4. 의사록_문장/문서 어조 라벨링

In [10]:
#의사록 문장 어조 라벨링

n_t_merge = pd.read_csv('./Project/total_label_matching.csv')
n_t_merge['tone_s']=''


for i in range(len(n_t_merge)):
    key1 = n_t_merge['매파count'][i]
    key2 = n_t_merge['비둘기파count'][i]
    
    if key1 != key2:
        
        subs = key1 - key2
        sums = key1 + key2

        n_t_merge['tone_s'][i] = subs / sums
        
    else:
        n_t_merge['tone_s'][i] = 0

n_t_merge.to_csv('./Project/total_label_matching_tone_s.csv', index = False)
n_t_merge.head()


/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,sec2_sec3_split,tokens,ngrams,ngrams and tokens,매파count,비둘기파count,tone_s
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...",3,1,0.5
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...,"환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/N...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...",1,2,-0.333333
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...,"유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...",1,3,-0.5
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...,"금리/NNG,수준/NNG,유지/NNG,되/XSV,금리/NNG,상승/NNG,하/XSV...","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/NNG","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/N...",1,2,-0.333333
4,2005.06.09,다른 일부 위원은 최근 가계대출의 월평균 증가액이 2003년과 지난해 실적 을 크...,"가계/NNG,대출/NNG,증가/NNG,액/NNG,실적/NNG,크/VA,넘/VV,주택...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN...","가계/NNG;대출/NNG;증가/NNG,주택담보대출/NNG;증가/NNG,주택가격/NN...",0,0,0


In [16]:

#문장어조를 기준으로 매파/비둘기파 라벨링
n_t_merge = n_t_merge.loc[n_t_merge['tone_s'] != 0]
n_t_merge['label'] = 0
n_t_merge['label'] = [1 if x>0 else -1 for x in n_t_merge['tone_s']]

n_t_merge.to_csv("./Project/total_label_matching_tone_s_label.csv", index = False)
n_t_merge.head()

,date,sec2_sec3_split,tokens,ngrams,ngrams and tokens,매파count,비둘기파count,tone_s,label
0,2005.06.09,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...","엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/...",3,1,0.5,1
1,2005.06.09,- 6 - 동 위원은 환율이 중장기적으로 각국의 기초경제여건(fundamen...,"환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/N...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...","원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일...",1,2,-0.333333,-1
2,2005.06.09,다른 일부 위원은 최근 유로지역의 경제성장 실적이 예상보다 저조한 가운 데 일각에...,"유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...","유로존/NNG;경제/NNG;성장/NNG,실적/NNG;예상/NNG;저조/NNG,전망/...",1,3,-0.5,-1
3,2005.06.09,동 위원은 국내 금리가 현 수준에서 유지되고 미국 금리가 상승하는 가운 데 달러화...,"금리/NNG,수준/NNG,유지/NNG,되/XSV,금리/NNG,상승/NNG,하/XSV...","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/NNG","자본/NNG;유출/NNG;확대/NNG,금리/NNG;상승/NNG,원화/NNG;강세/N...",1,2,-0.333333,-1
5,2005.06.09,이와 관련하여 또 다른 일부 위원은 일부 시중은행들이 저축은행들과 연계 하여 주택...,"또/MAG,저축은행/NNG,연계/NNG,하/XSV,주택/NNG,담보인정비율/NNG,...","담보인정비율/NNG;높/VV,대출/NNG;늘/VV","담보인정비율/NNG;높/VV,대출/NNG;늘/VV,또/MAG,저축은행/NNG,연계/...",0,1,-1,-1


In [17]:
#문서 매파/비둘기파 라벨링

import pandas as pd
tone_s = pd.read_csv("./Project/total_label_matching_tone_s_label.csv")
dd = tone_s.groupby(['date', 'label']).count().reset_index()
tone_tb = pd.merge(dd[dd.label==1][['date','tone_s']], dd[dd.label==-1][['date','tone_s']], on='date')
tone_tb['tone_i'] = 0


#tone_s_x : 해당 날짜에 매파 라벨링 된 문장 개수
#tone_s_y : 해당 날짜에 비둘기파 라벨링 된 문장 개수

tone_tb['tone_i'] = (tone_tb['tone_s_x']-tone_tb['tone_s_y'])/(tone_tb['tone_s_x']+tone_tb['tone_s_y'])

tone_tb.to_csv('./Project/tone_i_re.csv',index=False)
tone_tb.head()

,date,tone_s_x,tone_s_y,tone_i
0,2005.06.09,52,38,0.155556
1,2005.07.07,58,47,0.104762
2,2005.08.11,68,39,0.271028
3,2005.09.08,57,53,0.036364
4,2005.10.11,80,28,0.481481
